# Visualizing clusters of relationships between Countries-Indicators.

In [1]:
import pandas as pd
import json

from ipycytoscape import *
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from IPython.display import clear_output
import os

### Select the Country to study

In [2]:
path = '../PatternDetection/clusteringMeasures/'
model, th, cluster = 'ERMLP/', 'SemEP_40/', 'clusters/'
path_model = path+model
path_model_th = path+model+th
path_model_th_cls = path+model+th+cluster
prediction = pd.read_csv(path_model_th + 'prediction.txt', delimiter="\t", header=None)
prediction.columns = ['col1', 'col2', 'category', 'weight']

In [3]:
df_country = pd.read_csv('../Extracting_Data_Skynet/dataset/country_label.csv')
country_list = list(df_country.label_country.unique())
dict_country = dict(zip(df_country.label_country, df_country.country))

In [4]:
country_label1 = widgets.Combobox(
    # value='John',
    placeholder='Write a country name',
    options=country_list,
    description='Select Country:',
    ensure_option=True,
    disabled=False
)


button_cui1 = widgets.Button(description="Add Country")

def create_expanded_button(description, button_style):
    return widgets.Button(description=description, button_style=button_style,
                          layout=widgets.Layout(height='auto', width='auto'))

grid = GridspecLayout(1, 2, height='60px')
grid[0, 0] = country_label1 # label_cui1
grid[0, 1] = button_cui1

display(grid)

GridspecLayout(children=(Combobox(value='', description='Select Country:', ensure_option=True, layout=Layout(g…

In [5]:
def select_cluster(country):
    entries = os.listdir(path_model_th_cls)
    country_entries = []
    for name in entries:
        cls = pd.read_csv(path_model_th_cls + name, delimiter="\t", header=None)
        cls.columns = ['Country', 'Indicator', 'category', 'Cost']
        if country in cls.Country.to_list():
            country_entries.append(name)
    return country_entries
# entries = select_cluster('country:DEU')

### Select the Cluster to be visualized

In [6]:
dropdown_cluster = widgets.Dropdown(options = {}, description='Cluster:')
# dropdown_cluster.options=entries
display(dropdown_cluster)

Dropdown(description='Cluster:', options={}, value=None)

In [7]:
def button_cui1_clicked(parm):
    dropdown_cluster.options = select_cluster(dict_country[grid[0, 0].value])

grid[0, 1].on_click(button_cui1_clicked)

In [8]:
def adding_prediction(name, cls):
    c = name.split('-')[1][:-4]
    ix = prediction.index[prediction['col2'] == c].tolist()
    if len(ix)==0:
        return cls
    ix = ix[0]
    for ind in prediction.index[ix+1:]:
        if 'Cluster' == prediction['col1'][ind]:
            break
        cls.loc[len(cls.index)] = [prediction['col1'][ind], prediction['category'][ind], 'Predicted:'+prediction['col2'][ind],
                                   prediction['weight'][ind]]
    return cls

def load_cluster(name):
    cls = pd.read_csv(path_model_th_cls + name, delimiter="\t", header=None)
    cls.columns = ['Country', 'Indicator', 'category', 'Cost']
    cls = adding_prediction(name, cls)
    return cls

def create_json_to_cytoscape(name): # entity_type
    rdf_graph = load_cluster(name)
    graph_json = dict()
    graph_json['nodes'] = []
    graph_json['edges'] = []
    drug_id = dict()
    id_x = 0
    for i in range(rdf_graph.shape[0]):
        subj = rdf_graph.iloc[i]['Country']
        obj = rdf_graph.iloc[i]['Indicator']
        prop = rdf_graph.iloc[i]['category']
        edge = dict()
        edge['data'] = dict()

        if subj in drug_id.keys():
            edge['data']['id'] = id_x
            edge['data']['source'] = drug_id[subj]
            edge['data']['Label'] = prop
            id_x += 1
        else:
            node = dict()
            node['data'] = dict()
            drug_id[subj] = id_x
            node['data']['id'] = id_x
            node['data']['name'] = subj
            node['data']['type'] = 'Country' # entity_type[subj]
            edge['data']['id'] = id_x + 1
            edge['data']['source'] = id_x
            edge['data']['Label'] = prop
            graph_json['nodes'].append(node)
            id_x += 2
        if obj in drug_id.keys():
            edge['data']['target'] = drug_id[obj]
        else:
            node = dict()
            node['data'] = dict()
            drug_id[obj] = id_x
            node['data']['id'] = id_x
            node['data']['name'] = obj
            node['data']['type'] = 'Indicator' # entity_type[obj]
            edge['data']['target'] = id_x
            graph_json['nodes'].append(node)
            id_x += 1
#             if obj == k:
#                 node['classes'] = 'red'  # Single class

        graph_json['edges'].append(edge)

    return graph_json

def create_graph_cytoscape(name):
    middle_vertex = create_json_to_cytoscape(name)
    # load a style dictionary
    with open("styles_prediction.json") as fi:
        s = json.load(fi)
    # Create the cytoscape graph widget
    cytoscapeobj = CytoscapeWidget()
    cytoscapeobj.graph.add_graph_from_json(middle_vertex, directed=True, multiple_edges=True)  # , directed=True, input_data['elements']
    
    cytoscapeobj.set_style(s)
    cytoscapeobj.set_layout(name='dagre', animate=True, nodeSpacing = 5)  # concentric,  breadthfirst, fcose, dagre, grid
    return cytoscapeobj

### Visualize the cluster selected

In [9]:
button = widgets.Button(description="Visualize")
output_2 = widgets.Output()
display(button, output_2)

def on_button_clicked(tfry):
    with output_2:
        clear_output(wait=True)
#         graph_json = create_json_to_cytoscape(cls)
        display(create_graph_cytoscape(dropdown_cluster.value))

button.on_click(on_button_clicked)

Button(description='Visualize', style=ButtonStyle())

Output()

In [ ]:
# from google.colab import output
# output.enable_custom_widget_manager()